In [17]:
from selenium import webdriver
from selenium.webdriver.common.by import By
# from selenium.common.exceptions import NoSuchElementException, SessionNotCreatedException
from time import sleep
import pandas as pd
import time, json, os

In [8]:
# content from <p>
def get_content(driver,class_name):
    section_elements = driver.find_elements(By.CLASS_NAME, class_name)
    for div in section_elements:
        p_tags = div.find_elements(By.TAG_NAME, 'p')
        for p in p_tags:
            yield p.text

In [9]:
# <a> href attribute from <div>
def get_links_btn(driver,class_name):
    div_elements = []
    for c in class_name:
        div_elements += driver.find_elements(By.CLASS_NAME, c)
    for div in div_elements:
        a_tag = div.find_elements(By.TAG_NAME, 'a')
        for a in a_tag:
            yield a.get_attribute('href')

In [10]:
def get_num_last_page(driver,ul_class='pagination_page_number'):

    while (True):
        try:
            # Find the 'Next' button element by its class
            load_more_button = driver.find_element(By.XPATH, '//a[@aria-label="Go to next page"]')
            load_more_button.click()
            print("Clicked 'Next Page' button")
            sleep(5)
        except :
            # No more button element
            # get last number
            ul = driver.find_element(By.CLASS_NAME, ul_class)
            li_elements = ul.find_elements(By.TAG_NAME, 'li')
            last_page = int(li_elements[-1].text.strip())
            break

    return last_page

In [11]:
# get href each blogs from every pages number
def get_links_blogs(link,last_page):
    urls = []
    for num_page in range(last_page):
        page_url = f"{link}/{num_page+1}"
        for l in get_links_btn(page_url,'blog_card_header'):
            urls.append(l)
    return urls

In [12]:
def check_exist_class(driver,class_name):
    for name in class_name:
        try:
            element = driver.find_element(By.CLASS_NAME, name)
            if element:
                yield name
        except:
            pass

Footer

In [13]:
driver = webdriver.Safari()
driver.get("https://www.innovestx.co.th")

urls = []
h3_element = driver.find_elements(By.CSS_SELECTOR, 'h3.head')
# Click footer button and collect href
for h3 in h3_element:
    try:
        h3.click()
        print('click footer button')
        div_elements = driver.find_element(By.CLASS_NAME, 'footer_section_one')
        a_tag = div_elements.find_elements(By.TAG_NAME, 'a')
        for a in a_tag:
            urls.append(a.get_attribute('href'))
        sleep(2)
    except:
        pass

driver.quit()

click footer button
click footer button
click footer button
click footer button
click footer button


In [14]:
# start scraping
urls = list(set(urls))
visited_links = set()
n = 0

data = {
        "visited_links" : list(visited_links),
        "urls" : urls,
        "round" : n
    }

status_path = "InnovestX/InnovestX_status.json"

with open(status_path, "w") as json_file:
    json.dump(data, json_file, indent=4)
    
df = pd.DataFrame({'content':[],'link':[]})


Scraping Data

In [18]:
status_path = "InnovestX/InnovestX_status.json"
href_class = ['blog_card_header','invest_guide ','swiper-wrapper','inner-plan-inverse', 'blog_list','container','pagination_page_number']

with open(status_path, "r") as json_file:
    data = json.load(json_file)
    visited_links = set(data['visited_links'])
    links = data['urls']
    n = data['round']

In [19]:
csvfile_path = f'InnovestX/InnovestX_{n}records.csv'
start_time = time.time()

if os.path.exists(csvfile_path):
    df = pd.read_csv(csvfile_path)
else:
    df = pd.DataFrame({'content':[],'link':[]})

while links:
    # get url then pop out
    current_url = links.pop(0)
    content = " "
    
    if current_url in visited_links or current_url[:27] != "https://www.innovestx.co.th":
        continue

    print(f"Current URL: {current_url}")
    n += 1
    
    # Cannot scrape pdf file
    if current_url[:32] == "https://www.innovestx.co.th/docs":
        new_record = ["Document", current_url]
        df.loc[len(df.index)] = new_record
        continue

    visited_links.add(current_url)

    driver = webdriver.Safari()
    driver.get(current_url)

    exist_class = list(check_exist_class(driver,href_class))
    
    # Block urls (has number label)
    if "pagination_page_number" in exist_class:
        try:
            last_page = get_num_last_page(driver)
            links += get_links_blogs(driver,last_page)
            print(f"Last pages: {last_page}")
        except :
            pass
    
    # Block urls (has not number label)
    try:
        links += list(get_links_btn(driver,exist_class))
    except:
        pass
    
    # get content from <p>
    if 'container' in exist_class:
        content = " ".join([text.strip() for text in get_content(driver,'container')])

    links = list(set(links))
    new_record = [content, current_url]
    df.loc[len(df.index)] = new_record

    driver.quit()

    print(f"Remaining urls: {len(links)}")
    print(f"Visited urls: {len(visited_links)}\n")

    sleep(5)
    
    if not n%20:
        data = {
            "visited_links" : list(visited_links),
            "links" : links
        }
        with open(status_path, "w") as json_file:
            json.dump(data, json_file, indent=4)
         
        elapsed_time = time.time() - start_time
        csvfile_path = f'InnovestX/InnovestX_{n}records.csv'
        print(f'write file using {(elapsed_time/60):.2f} minutes')
        df.to_csv(csvfile_path, index=False, encoding='utf-8-sig')


Current URL: https://www.innovestx.co.th/invest-with-innovestx
Remaining urls: 47
Visited urls: 1

Current URL: https://www.innovestx.co.th/innovestx-video/detail/innovestx-vdo/guru7
Remaining urls: 63
Visited urls: 2

Current URL: https://www.innovestx.co.th/promotions/detail/mutual-fund-mega-sale
Remaining urls: 63
Visited urls: 3

Current URL: https://www.innovestx.co.th/trader-camp
Remaining urls: 63
Visited urls: 4

Current URL: https://www.innovestx.co.th/innovestx-video/detail/innovestx-vdo/guru8
Remaining urls: 65
Visited urls: 5

Current URL: https://www.innovestx.co.th/point-to-invest-promotions
Remaining urls: 65
Visited urls: 6

Current URL: https://www.innovestx.co.th/point-to-invest-promotions#
Remaining urls: 65
Visited urls: 7

Current URL: https://www.innovestx.co.th/news
Remaining urls: 73
Visited urls: 8

Current URL: https://www.innovestx.co.th/innovestx-video/detail/innovestx-vdo/guruofthegalaxy1
Remaining urls: 76
Visited urls: 9

Current URL: https://www.innovest

KeyboardInterrupt: 